In [1]:
from data import BinanceAPI
import pandas as pd

/Users/baobach/anaconda3/envs/pythonlab/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_directory" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [2]:
binance = BinanceAPI()

In [3]:
binance.minute_bar(symbol='BTCUSDT',begin = '1 Jan 2018', end='1 Jan 2023')

Downloading...
Completed!


In [12]:
df = pd.read_parquet('./data/BTCUSDT_1Min')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2621433 entries, 2018-01-01 00:00:00 to 2023-01-01 00:00:00
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   open    object
 1   high    object
 2   low     object
 3   close   object
 4   volume  object
dtypes: object(5)
memory usage: 120.0+ MB


In [14]:
# Specify column to convert
columns_to_convert = ['open', 'high', 'low', 'close', 'volume']

# Convert specified columns to float64
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Verify the data types after conversion
print(df.dtypes)


open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object


In [16]:
df['v'] = df['volume']
df['dv'] = df['volume']*df['close']
df.head()

,open,high,low,close,volume,v,dv
timestamp,,,,,,,
2018-01-01 00:00:00,13715.65,13715.65,13681.00,13707.92,2.844266,2.844266,38988.970787
2018-01-01 00:01:00,13707.91,13707.91,13666.11,13694.92,2.113138,2.113138,28939.255859
2018-01-01 00:02:00,13682.00,13694.94,13680.00,13680.00,1.346411,1.346411,18418.902480
2018-01-01 00:03:00,13679.98,13679.98,13601.00,13645.99,11.589417,11.589417,158149.068488
2018-01-01 00:04:00,13645.98,13646.00,13576.28,13600.00,15.724566,15.724566,213854.097600
